In [2]:
#Importing the required libraries
from dotenv import load_dotenv
load_dotenv()
from pinecone import Pinecone, ServerlessSpec
import google.generativeai as genai
import os
import json

In [3]:
#Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [4]:
#Create a Pinecone index
pc.create_index(
    name = "rmp-ai-assistant",
    dimension = 768,
    metric = "cosine",
    spec = ServerlessSpec(cloud="aws", region="us-east-1"),
)

In [5]:
#Load the review data
data = json.load(open("reviews.json"))

In [6]:
processed_data = []

In [7]:
#Initialize Gemini Pro
genai.configure(api_key=os.environ["GEMINI_API_KEY"])
client = genai.GenerativeModel('gemini-1.5-flash')

In [9]:
#Create embeddings for each review
model = 'models/embedding-001'
for review in data["reviews"]:
    embedding = genai.embed_content(model=model, content=review["review"], task_type="retrieval_document")
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"],
        }
    })

In [10]:
#Insert the embedding into the Pinecone index
index = pc.Index("rmp-ai-assistant")
upsert_response = index.upsert(vectors=processed_data, namespace="ns1")
print(f"Upserted count: {upsert_response['upserted_count']}")

ListConversionException: Expected a list or list-like data structure, but got: {'embedding': [0.07360755, -0.013527036, 0.004877886, -0.039610516, 0.028022688, 0.05607909, 0.020386362, -0.006763031, 0.0055529457, 0.03830611, -0.059646048, -0.02801522, 0.003307147, 0.03074215, -0.0005718069, -0.015071498, 0.02667722, -0.010668662, -0.010267779, -0.014468648, 0.0028611862, -0.006737254, 0.053965792, -0.046402633, -0.0020500058, -0.030388232, 0.01120867, -0.036938727, 0.008256921, -0.003492913, -0.06419695, 0.005049514, -0.034901764, 0.046862617, 0.0013944953, -0.029306242, 0.0072802724, -0.03386555, -0.023613282, 0.036619082, 0.020055179, -0.035372913, -0.03891479, 0.04234849, 0.0037250323, 0.0049976227, -0.011027548, 0.012246794, 0.0064763804, -0.031678524, 0.017794387, 0.041887328, 0.021047497, -0.002032083, 0.025198085, -0.02814984, 0.0589758, -0.04171055, -0.020720815, 0.020454837, -0.010793376, 0.005636296, 0.0068745, -0.012301323, 0.0032522958, -0.07045457, -0.043046836, 0.00777218

In [ ]:
#Print index statistics
print(index.describe_index_stats())